In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
import inspect
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3


### Get recording info (google sheets)

In [3]:
import requests
import pandas as pd
from io import BytesIO

In [4]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [5]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,calibration_board_shape,calibration_square_size,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,NaN,NaN,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1


### Submit job

In [6]:
from multicamera_airflow_pipeline.tim_240731.interface.o2 import O2Runner
from pathlib import Path
from datetime import datetime

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [7]:
for idx, recording_row in recording_df.iterrows():
    break

In [8]:
recording_row

Subject                                                                  M04002
duration_m                                                                   10
video_recording_id                                     24-05-01-13-26-43-110846
ephys_id                                                    2024-05-01_13-26-37
calibration_id                                         24-05-01-13-45-07-825493
calibration_board_shape                                                     NaN
calibration_square_size                                                     NaN
video_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
ephys_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
calibration_location_on_o2    /n/groups/datta/tim_sainburg/datasets/chronic2...
samplerate                                                                  150
username                                                                 tis697
n_ephys_streams                         

### Job specific

In [9]:
import yaml
import textwrap
import time

In [10]:
# these are arguments
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240808-3d-pipeline"
job_directory = Path('/n/groups/datta/tim_sainburg/datasets/scratch/jobs')
job_directory.mkdir(exist_ok=True, parents=True)
config_file = Path("/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml")

In [11]:
def convert_minutes_to_hms(minutes_float):
    # Convert minutes to total seconds
    total_seconds = int(minutes_float * 60)
    
    # Extract hours, minutes, and seconds using divmod
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    # Format as HH:MM:SS
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [12]:
def check_ephys_sync_completion(output_directory_ephys_sync, n_expected_streams):
    # TODO check if number of ephys streams matches expected
    n_aligned_ephys_streams = len(list(output_directory_ephys_sync.glob("*/ephys_alignment.mmap")))
    if n_aligned_ephys_streams == n_expected_streams:
        return True
    return False

In [13]:
n_expected_streams = recording_row.n_ephys_streams

In [15]:
# load config
config_file = Path(config_file)
config = yaml.safe_load(open(config_file, "r"))

In [16]:
# where the video data is located
recording_directory = (
    Path(recording_row.video_location_on_o2) / recording_row.video_recording_id
)

In [17]:
# where to save output
output_directory_ephys_sync = (
    output_directory / "openephys_sync" / recording_row.video_recording_id
)
output_directory_ephys_sync.mkdir(parents=True, exist_ok=True)
current_datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
remote_job_directory = job_directory / current_datetime_str

In [23]:
# check if sync successfully completed
if config["sync_ephys"]["recompute_completed"] == False:
    if check_ephys_sync_completion(output_directory_ephys_sync, n_expected_streams) == False:
        print('No streams matched')
        # return

No streams matched


In [24]:
samplerate = recording_row.samplerate

In [25]:
recording_row.ephys_location_on_o2

'/n/groups/datta/tim_sainburg/datasets/chronic24/ephys/M04002/'

In [26]:
camera_sync_file = output_directory  / 'camera_sync' / recording_row.video_recording_id / 'camera_sync.csv'
assert camera_sync_file.exists()

In [27]:
ephys_recording_path = Path(recording_row.ephys_location_on_o2) / recording_row.ephys_id
assert ephys_recording_path.exists()

In [28]:
params = {
    "recording_directory": recording_directory.as_posix(),
    "ephys_sync_output_directory": output_directory_ephys_sync.as_posix(),
    "samplerate": samplerate,
    "camera_sync_file": camera_sync_file.as_posix(),
    "ephys_recording_path": ephys_recording_path.as_posix()
}

In [29]:
duration_requested = convert_minutes_to_hms(
        recording_row.duration_m * config["o2"]["sync_ephys"]["o2_runtime_multiplier"]
    )
duration_requested

'00:10:00'

In [30]:
# create the job runner
runner = O2Runner(
    job_name_prefix=f"{recording_row.video_recording_id}_ephys_sync",
    remote_job_directory=remote_job_directory,
    conda_env="/n/groups/datta/tim_sainburg/conda_envs/peromoseq",
    o2_username=recording_row.username,
    o2_server="login.o2.rc.hms.harvard.edu",
    job_params=params,
    o2_n_cpus=config["o2"]["sync_ephys"]["o2_n_cpus"],
    o2_memory=config["o2"]["sync_ephys"]["o2_memory"],
    o2_time_limit=duration_requested,
    o2_queue=config["o2"]["sync_ephys"]["o2_queue"],
)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Auth banner: b'Problems logging in?\nUse your lower case HMS ID, like abc123, not ABC123.\nIf locked out, see:\nhttps://it.hms.harvard.edu/i-want/reset-password-or-unlock-your-hms-account\n'
INFO:paramiko.transport:Authentication (publickey) successful!


In [31]:
runner.python_script = textwrap.dedent(
f"""
# load params
import yaml
params_file = "{runner.remote_job_directory / f"{runner.job_name}.params.yaml"}"
config_file = "{config_file.as_posix()}"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

# grab sync cameras function
from multicamera_airflow_pipeline.tim_240731.sync.sync_cameras_to_openephys import OpenEphysSynchronizer
synchronizer = OpenEphysSynchronizer(
    camera_sync_file =params["camera_sync_file"] ,
    ephys_recording_path = params["ephys_recording_path"],
    ephys_sync_output_directory =  params["ephys_sync_output_directory"],
    camera_samplerate = params["samplerate"],
    **config["sync_ephys"]
)
synchronizer.run()
"""
)

In [32]:
print(runner.python_script)


# load params
import yaml
params_file = "/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102704_162068/24-05-01-13-26-43-110846_ephys_sync_24-08-09-2024-27-22-297432.params.yaml"
config_file = "/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

# grab sync cameras function
from multicamera_airflow_pipeline.tim_240731.sync.sync_cameras_to_openephys import OpenEphysSynchronizer
synchronizer = OpenEphysSynchronizer(
    camera_sync_file =params["camera_sync_file"] ,
    ephys_recording_path = params["ephys_recording_path"],
    ephys_sync_output_directory =  params["ephys_sync_output_directory"],
    camera_samplerate = params["samplerate"],
    **config["sync_ephys"]
)
synchronizer.run()



In [33]:
runner.run()

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote job directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102704_162068
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102704_162068
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Successfully created remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102704_162068
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Writing job files to remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102704_162068
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Transferring /tmp/tmptehqaq5g to login.o2.rc.hms.harvard.edu:/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102704_162068/24-05-01-13-26-43-110846_ephys_sync_24-08-09-2024-27-22-297432.

In [34]:
# wait until the job is finished
# 10000/60/24 = roughly 1 week
for i in range(10000):
    # check job status every n seconds
    status = runner.check_job_status()
    if status:
        break
    time.sleep(60)

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43723637
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is waiting to be scheduled.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43723637
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job has finished successfully.


In [36]:
# check if sync successfully completed
if check_ephys_sync_completion(output_directory_ephys_sync, n_expected_streams):
    print('Ephys sync completed successfully')
else:
    raise ValueError("Ephys sync did not complete successfully.")


In [47]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102039_489984/

24-05-01-13-26-43-110846_ephys_sync_24-08-09-2024-20-39-764709.log
24-05-01-13-26-43-110846_ephys_sync_24-08-09-2024-20-39-764709.params.yaml
24-05-01-13-26-43-110846_ephys_sync_24-08-09-2024-20-39-764709.py
24-05-01-13-26-43-110846_ephys_sync_24-08-09-2024-20-39-764709.sh


In [48]:
!tail /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102039_489984/24-05-01-13-26-43-110846_ephys_sync_24-08-09-2024-20-39-764709.log

Traceback (most recent call last):
  File "/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102039_489984/24-05-01-13-26-43-110846_ephys_sync_24-08-09-2024-20-39-764709.py", line 19, in <module>
    synchronizer.run()
  File "/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/sync/sync_cameras_to_openephys.py", line 100, in run
    self.load_camera_sync()
  File "/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/sync/sync_cameras_to_openephys.py", line 66, in load_camera_sync
    if self.check_if_camera_sync_exists() == False:
  File "/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/sync/sync_cameras_to_openephys.py", line 46, in check_if_camera_sync_exists
    if self.camera_sync_file.exists():
AttributeError: 'str' object has no attribute 'exists'
